### DQN Performance and Hedging Behaviour - EDA 

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from dqn_per import DQN
from envs import TradingEnv


### Training reward and Loss. 

In [ ]:
train_num = 25
df = pd.read_csv(f'history\dqn\Training{train_num}\dqn_training_history.csv')

#Smoothing function for training data 
def smooth(series, window=1000):
    return series.rolling(window=window, min_periods=1).mean()

df["reward_smooth"] = smooth(df["reward"])
df["loss_smooth"] = smooth(df["loss"])


#Plotting training data 
fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharex=True)

ax[0].plot(df["episode"], df["reward_smooth"], color = "green")
ax[0].set_title("Smoothed Reward per Episode")
ax[0].set_ylabel("Average Reward")
ax[0].set_xlabel("Episode")

ax[1].plot(df["episode"], df["loss_smooth"], color = "red")
ax[1].set_title("Smoothed Loss per Episode")
ax[1].set_ylabel("Average Loss")
ax[1].set_xlabel("Episode")

plt.tight_layout()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Reward_Loss_Episodes.png')
plt.show()


### Loading in Environment and model

In [ ]:
oos =10000
cost_multiplier = 0
kappa = 1/10
env = TradingEnv(continuous_action_flag=False, sabr_flag=False, 
                    dg_random_seed= 1, spread=0.0, num_contract=1, 
                    init_ttm=10, trade_freq=1/5, num_sim=oos,kappa= kappa,cost_multiplier=cost_multiplier,
                        mu =0, vol =0.01* np.sqrt(250), S = 100, K = 100, r = 0, q = 0)
dqn = DQN(env)

    # Load the trained model
dqn.load(path = f"model/dqn/Training{train_num}/dqn_model_282000.h5")

episode_rewards, dqn_actions, q_values = dqn.test(total_episode=oos)
dqn_actions =np.array(dqn_actions)
dqn_actions = np.insert(dqn_actions, 0, 0, axis=1)
action = np.diff(dqn_actions, axis=1)

In [ ]:
# Select an episode to visualize
episode_idx = 73  # Change this to the desired episode index

# Extract Q-values, DQN actions, and delta hedging actions for the selected episode
q_values_episode = np.array(q_values[episode_idx])  # Shape: (time_steps, num_actions)
actions_episode = dqn_actions[episode_idx, 1:]  # Shape: (time_steps,)
delta_actions_episode = env.delta_path[episode_idx, 1:] * 100  # Scale delta actions

# Create the heatmap
# Create the heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(q_values_episode.T, cmap="RdYlGn", annot=False, cbar_kws={'label': 'Q-value'})

# Overlay the chosen DQN actions
time_steps = np.arange(1,q_values_episode.shape[0]+1)
plt.plot(time_steps, actions_episode, 'k.', label="DQN Action", markersize=8)

# Overlay the delta hedging actions
plt.plot(time_steps, delta_actions_episode, 'r--', color="orange", label="Delta Hedging Action", linewidth=1.5)

# Add labels and title
plt.xlabel("Time Step", fontsize=12, labelpad=10)
plt.ylabel("Action Index", fontsize=12, labelpad=10)
plt.title(f"Q-value Heatmap with DQN and Delta Hedging Actions (Episode {episode_idx})", fontsize=14, pad=15)

# Set axis limits
plt.xlim(1, 50)  # X-axis from time step 1 to 50
plt.ylim(0, 100)  # Y-axis from 0 to 100

# Add legend and save the plot
plt.legend(fontsize=10)
plt.tight_layout()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Qvalues_Actions_Episode{episode_idx}.png')
plt.show()

### Transaction Costs KDE

In [ ]:
# Function to calculate transaction costs based on delta hedging
def cost(delta_h, multiplier):
    TickSize = 0.1
    return multiplier * TickSize * (np.abs(delta_h) + 0.01 * delta_h**2)


#Calculating the transaction costs for the DQN Policy 
transaction_costs_dqn = cost(action, multiplier=cost_multiplier)
total_costs_dqn = np.sum(transaction_costs_dqn, axis =1)

#Calculating the transaction costs for the delta policy
delta_path = env.delta_path*100
delta_path_append = np.insert(delta_path, 0, 0, axis=1)
delta_path.shape

delta_h = np.diff(delta_path_append, axis=1)
delta_h= delta_h[:, :-1]
transaction_costs_delta= cost(delta_h, multiplier=cost_multiplier)
total_costs_delta = np.sum(transaction_costs_delta, axis =1)


sns.set_style("whitegrid")
# Plot histogram for the distribution of total hedging costs
plt.figure(figsize=(10, 5))
sns.kdeplot(total_costs_dqn,  shade = False, color="green", label='DQN Hedging Costs', bw_adjust=3)
sns.kdeplot(total_costs_delta, shade = False, color="orange", label='Delta Hedging Costs', bw_adjust=3)
plt.xlim(0, )
plt.xlabel("Total Hedging Cost")
plt.ylabel("Frequency")
plt.title("Distribution of Total Hedging Costs")
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Hedging_Costs_Distribution.png')
plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import t, f
import numpy as np

# Data
avg_gamma = np.average(env.gamma_path, axis=1)
total_costs = np.sum(transaction_costs_dqn, axis=1)

X = avg_gamma.reshape(-1, 1)
y = total_costs

# Fit the regression model
model = LinearRegression()
model.fit(X, y)

# Predictions
y_pred = model.predict(X)

# Residuals
residuals = y - y_pred

# Degrees of freedom
n = len(y)  # Number of observations
p = X.shape[1] + 1  # Number of predictors + intercept
df_residual = n - p  # Residual degrees of freedom
df_model = p - 1  # Model degrees of freedom

# Standard error of residuals
residual_std_error = np.sqrt(np.sum(residuals**2) / df_residual)

# Coefficients and standard errors
X_with_intercept = np.hstack([np.ones((X.shape[0], 1)), X])  # Add intercept
coef_cov_matrix = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * residual_std_error**2
std_errors = np.sqrt(np.diag(coef_cov_matrix))

# t-values and p-values for coefficients
t_values = model.coef_ / std_errors[1:]
t_values = np.insert(t_values, 0, model.intercept_ / std_errors[0])  # Add intercept t-value
p_values = [2 * (1 - t.cdf(np.abs(t_val), df_residual)) for t_val in t_values]

# R-squared and adjusted R-squared
r_squared = r2_score(y, y_pred)
adj_r_squared = 1 - (1 - r_squared) * (n - 1) / df_residual

# F-statistic and p-value
ssr = np.sum((y_pred - np.mean(y))**2)  # Regression sum of squares
sse = np.sum(residuals**2)  # Sum of squared errors
msr = ssr / df_model  # Mean square regression
mse = sse / df_residual  # Mean square error
f_stat = msr / mse  # F-statistic
f_p_value = 1 - f.cdf(f_stat, df_model, df_residual)

# Summary
summary = f"""
Call:
LinearRegression(formula = total_costs ~ avg_gamma)

Residuals:
    Min      1Q  Median      3Q     Max 
{np.min(residuals):.3f}  {np.percentile(residuals, 25):.3f}  {np.median(residuals):.3f}  {np.percentile(residuals, 75):.3f}  {np.max(residuals):.3f}

Coefficients:
            Estimate    Std. Error    t value    Pr(>|t|)    
(Intercept) {model.intercept_:.5f}   {std_errors[0]:.5f}   {t_values[0]:.2f}   {p_values[0]:.2e}    
avg_gamma   {model.coef_[0]:.5f}   {std_errors[1]:.5f}   {t_values[1]:.2f}   {p_values[1]:.2e}    

Residual standard error: {residual_std_error:.3f} on {df_residual} degrees of freedom
Multiple R-squared:  {r_squared:.4f}, Adjusted R-squared:  {adj_r_squared:.4f}
F-statistic: {f_stat:.2f} on {df_model} and {df_residual} DF,  p-value: {f_p_value:.2e}
"""

print(summary)

# Plot the regression line
plt.figure(figsize=(10, 6))
plt.scatter(X, y, color='green', alpha=0.4, label='Data Points')
plt.plot(X, y_pred, color='purple', linewidth=2, label='Regression Line')
plt.xlabel('Average Gamma')
plt.ylabel('Total Hedging Cost')
plt.yticks(ticks=np.arange(0, np.max(y), step=10), labels=np.arange(0, np.max(y), step=10))
plt.title('Regression of Total Hedging Cost of DQN agent on Average Gamma')
plt.legend()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Hedging_Costs_Regression.png')
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Apply polynomial transformation
poly = PolynomialFeatures(degree=2)  # Try degree=2 or higher
X_poly = poly.fit_transform(avg_gamma.reshape(-1, 1))

# Fit the polynomial regression model
model_poly = LinearRegression()
model_poly.fit(X_poly, total_costs)

# Predictions
y_pred_poly = model_poly.predict(X_poly)

# Evaluate the fit
r_squared_poly = r2_score(total_costs, y_pred_poly)
print(f"R-squared (Polynomial): {r_squared_poly:.4f}")

## Total Profit & Loss t-statistic and volatility

In [ ]:
# option value V_t 
v_t = env.option_price_path*100
v_t_diff = np.diff(v_t, axis=1)

# hedged position value 
s_t = env.path
s_t_diff = np.diff(s_t, axis=1)

# total pnl_t BS 
a_t_delta = delta_path[:, :-1]
h_t_delta = a_t_delta*s_t_diff
pi_t_delta = v_t_diff - h_t_delta - transaction_costs_delta
total_pnl_delta = np.sum(pi_t_delta[:, 1:], axis = 1)

# total pnl_t DQN
a_t_dqn = dqn_actions[:, :-1]
h_t_dqn = a_t_dqn * s_t_diff
pi_t_dqn = v_t_diff - h_t_dqn - transaction_costs_dqn
total_pnl_dqn = np.sum(pi_t_dqn[:,1:], axis = 1)

sns.set_style("whitegrid")
# Plot histogram for the distribution of total hedging costs
plt.figure(figsize=(10, 5))
sns.kdeplot(total_pnl_delta, color="orange", label='Delta Hedging PnL')
sns.kdeplot(total_pnl_dqn, color="green", label ='DQN Hedging PnL')
plt.legend()
plt.xlabel("Total Hedging PnL")
plt.ylabel("Frequency")
plt.title("Distribution of Total PnL")
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Hedging_PnL_Distribution.png')
plt.show()



In [ ]:
# Calculate statistical measures for total_pnl_dqn
mean_pnl = np.mean(total_pnl_dqn)
std_pnl = np.std(total_pnl_dqn, ddof=1)
var_level = 0.05  # 5% VaR
var_95 = np.percentile(total_pnl_dqn, 100 * var_level)
cvar_95 = total_pnl_dqn[total_pnl_dqn <= var_95].mean()
sharpe_ratio = mean_pnl / std_pnl if std_pnl != 0 else np.nan

# Save results to CSV
stats = {
    "Mean PnL": mean_pnl,
    "Std PnL": std_pnl,
    "5% VaR": var_95,
    "5% CVaR": cvar_95,
    "Sharpe Ratio": sharpe_ratio
}
stats_df = pd.DataFrame([stats])
stats_df.to_csv(f'history/dqn/Training{train_num}/Training{train_num}_DQN_PnL_Stats.csv', index=False)

print(stats_df.round(4))

In [ ]:
#metrics for delta hedging
mean_pnl_delta = np.mean(total_pnl_delta)
std_pnl_delta = np.std(total_pnl_delta, ddof=1)
var_level = 0.05  # 5% VaR
var_95_delta = np.percentile(total_pnl_delta, 100 * var_level)
cvar_95_delta = total_pnl_delta[total_pnl_delta <= var_95_delta].mean()
sharpe_ratio_delta = mean_pnl_delta / std_pnl_delta if std_pnl_delta != 0 else np.nan

# Save results to CSV
stats = {
    "Mean PnL": mean_pnl_delta,
    "Std PnL": std_pnl_delta,
    "5% VaR": var_95_delta,
    "5% CVaR": cvar_95_delta,
    "Sharpe Ratio": sharpe_ratio_delta
}
stats_df_delta = pd.DataFrame([stats])
stats_df_delta.to_csv(f'history/dqn/Training{train_num}/Training{train_num}_Delta_PnL_Stats.csv', index=False)

print(stats_df_delta.round(4))

In [ ]:
# Create a heatmap of the correlation between DQN actions, delta, and gamma for all episodes

# Arrays to store correlations for each episode
corr_matrix = []

for ep in range(dqn_actions.shape[0]):
    dqn_act = abs(action[ep,:])
    delta_act = abs(delta_h[ep,:])
    gamma = env.gamma_path[ep, 1:]
    # Compute pairwise correlations
    corr_ep = np.corrcoef([dqn_act, delta_act, gamma])
    corr_matrix.append(corr_ep)

# Convert to numpy array and average across episodes
corr_matrix = np.array(corr_matrix)  # shape: (episodes, 3, 3)
mean_corr = np.mean(corr_matrix, axis=0)

# Plot heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(mean_corr, annot=True, cmap="coolwarm", xticklabels=["DQN", "Delta", "Gamma"], yticklabels=["DQN", "Delta", "Gamma"])
plt.title("Mean Correlation Heatmap: DQN, Delta, Gamma (across episodes)")
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_CorrHeatMap_DQN_Delta_Gamma.png')
plt.show()

In [ ]:
better_delta_idx = np.where(total_pnl_delta > total_pnl_dqn)[0]
better_dqn_idx = np.where(total_pnl_dqn >= total_pnl_delta)[0]
print(f"Number of episodes where Delta outperforms DQN: {len(better_delta_idx)}")

# Visualize

plt.figure(figsize=(10, 5))
plt.hist(total_pnl_delta - total_pnl_dqn, bins=50, color='gray')
plt.axvline(0, color='blue', linestyle='--', label='Delta = DQN')
plt.xlabel("Delta PnL - DQN PnL")
plt.ylabel("Episode Count")
plt.title("Episodes: Delta Hedging Outperforms DQN")
plt.legend()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Delta_vs_DQN_PnL.png')
plt.show()

In [ ]:
episode_idx = better_dqn_idx[200]  # Change this to the desired episode index

# Extract Q-values and actions for the selected episode
q_values_episode = np.array(q_values[episode_idx])  # Shape: (time_steps, num_actions)
actions_episode = dqn_actions[episode_idx, 1:]  # Shape: (time_steps,)
delta_actions_episode = env.delta_path[episode_idx, 1:] * 100

# Create the heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(q_values_episode.T, cmap="RdYlGn", annot=False, cbar_kws={'label': 'Q-value'})

# Overlay the chosen actions
time_steps = np.arange(q_values_episode.shape[0])
plt.plot(time_steps, actions_episode, 'k.', label="DQN Action", markersize=8)

# Overlay the delta hedging actions
plt.plot(time_steps, delta_actions_episode, 'r--', label="Delta Hedging Action", linewidth=1.5)

# Add labels and title
plt.xlabel("Time Step", fontsize=12, labelpad=10)
plt.ylabel("Action Index", fontsize=12, labelpad=10)
plt.title(f"Q-value Heatmap with DQN and Delta Hedging Actions (Episode {episode_idx})", fontsize=14, pad=15)
plt.legend(fontsize=10)
plt.tight_layout()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_QValue_Heatmap_Episode_{episode_idx}.png')
plt.show()

stock_price_episode = env.path[episode_idx]

# Plot stock price movement
plt.figure(figsize=(7, 5))
plt.plot(stock_price_episode, label="Stock Price", color="blue")
plt.xlabel("Time Step", fontsize=12, labelpad=10)
plt.ylabel("Stock Price", fontsize=12, labelpad=10)
plt.title(f"Stock Price Movement (Episode {episode_idx})", fontsize=14, pad=15)
plt.legend(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_StockPrice_Episode_{episode_idx}.png')
plt.show()

colors = {
    "dqn_error": "#2ca02c",  # Green
    "delta_error": "#ff7f0e",  # Orange
    "dqn_cost": "#1f77b4",  # Blue
    "delta_cost": "#d62728",  # Red
}

# Create the combined plot
plt.figure(figsize=(7, 5))

# Plot hedging error
plt.plot(v_t_diff[episode_idx] - h_t_dqn[episode_idx], label="DQN Hedging Error", color=colors["dqn_error"], linestyle="--")
plt.plot(v_t_diff[episode_idx] - h_t_delta[episode_idx], label="Delta Hedging Error", color=colors["delta_error"], linestyle="--")

# Plot hedging cost
plt.plot(-transaction_costs_dqn[episode_idx], label="DQN Hedging Cost", color=colors["dqn_cost"], linestyle="-")
plt.plot(-transaction_costs_delta[episode_idx], label="Delta Hedging Cost", color=colors["delta_cost"], linestyle="-")

# Add labels, title, and legend
plt.xlabel("Time Step", fontsize=12, labelpad=10)
plt.ylabel("Value", fontsize=12, labelpad=10)
plt.title(f"Hedging Error and Cost (Episode {episode_idx})", fontsize=14, pad=15)
plt.legend(fontsize=10, loc='upper left')  # Adjust legend position
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Hedging_Error_Cost_Episode_{episode_idx}.png')
plt.show()

In [ ]:
# Calculate terminal portfolio values (last time step PnL)
terminal_pnl_dqn = pi_t_dqn[:, -1]
terminal_pnl_delta = pi_t_delta[:, -1]

plt.figure(figsize=(10, 5))
sns.kdeplot(terminal_pnl_dqn, label="DQN", color="green", bw_adjust=1.2)
sns.kdeplot(terminal_pnl_delta, label="Delta Hedge", color="orange", bw_adjust=1.2)
plt.xlabel("Terminal Portfolio Value")
plt.ylabel("Density")
plt.title("Distribution of Terminal Portfolio Value")
plt.legend()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Terminal_PnL.png')
plt.show()

In [ ]:
plt.figure(figsize=(7, 6))
plt.hexbin(terminal_pnl_dqn, terminal_pnl_delta, gridsize=50, cmap='viridis', mincnt=1)
plt.plot([min(terminal_pnl_dqn), max(terminal_pnl_dqn)],
         [min(terminal_pnl_dqn), max(terminal_pnl_dqn)], 'r--', label='45-degree line')
plt.xlabel("DQN Terminal Portfolio Value")
plt.ylabel("Delta Hedge Terminal Portfolio Value")
plt.title("Hexbin: DQN vs Delta Terminal Portfolio Value")
plt.colorbar(label='Count')
plt.legend()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Hexbin_DQN_Delta.png')
plt.show()

In [ ]:
std_pnl_dqn = np.std(pi_t_dqn, axis=1)
std_pnl_delta = np.std(pi_t_delta, axis=1)

plt.figure(figsize=(10, 5))
sns.kdeplot(std_pnl_dqn, label="DQN", color="green", bw_adjust=1.2)
sns.kdeplot(std_pnl_delta, label="Delta Hedge", color="orange", bw_adjust=1.2)
plt.xlabel("Std Dev of PnL within Episode")
plt.ylabel("Density")
plt.title("Distribution of Std Dev of PnL within Episode")
plt.legend()
plt.show()

In [ ]:
# Compute average std dev in each hexbin
plt.figure(figsize=(7, 6))
hb = plt.hexbin(terminal_pnl_dqn, terminal_pnl_delta, C=std_pnl_dqn, 
                gridsize=50, cmap='viridis', reduce_C_function=np.mean, mincnt=1)
plt.xlabel("DQN Terminal Portfolio Value")
plt.ylabel("Delta Hedge Terminal Portfolio Value")
plt.title("Hexbin: Terminal Value, Colored by Avg Std Dev (DQN)")
cb = plt.colorbar(hb)
cb.set_label('Avg Std Dev of DQN PnL')
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Hexbin_Std_dev.png')
plt.show()

In [ ]:
# ...existing code...

percentiles = [5, 25, 50, 75, 95]
pnl_percentiles = np.percentile(pi_t_dqn, percentiles, axis=0)

plt.figure(figsize=(10, 5))
x = np.arange(pnl_percentiles.shape[1])

# Shade between 5th and 95th percentile (light blue)
plt.fill_between(x, pnl_percentiles[0], pnl_percentiles[-1], color='#ADD8E6', alpha=0.5)

# Shade between 25th and 75th percentile (light yellow)
plt.fill_between(x, pnl_percentiles[1], pnl_percentiles[3], color='#FFFACD', alpha=0.7)

# Median line (50th percentile, bold)
plt.plot(x, pnl_percentiles[2], color='blue', linewidth=2, label='Median (50th percentile)')

# Percentile lines
plt.plot(x, pnl_percentiles[0], color='#1E90FF', linestyle='--', linewidth=1, label='5th percentile')
plt.plot(x, pnl_percentiles[1], color='#FFD700', linestyle='--', linewidth=1, label='25th percentile')
plt.plot(x, pnl_percentiles[3], color='#FFD700', linestyle='--', linewidth=1, label='75th percentile')
plt.plot(x, pnl_percentiles[4], color='#1E90FF', linestyle='--', linewidth=1, label='95th percentile')

plt.xlabel("Time Step", fontsize=13)
plt.ylabel("PnL", fontsize=13)
plt.title("DQN PnL Distribution Over Time", fontsize=15)
plt.legend(loc='upper right', fontsize=11, frameon=True)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_PnL_Percentile_Shaded.png', dpi=300)
plt.show()

In [ ]:
abs_diff = np.abs(pi_t_dqn) - np.abs(pi_t_delta)
mean_abs_diff = np.mean(abs_diff, axis=0)
percentiles_diff = np.percentile(abs_diff, [5, 25, 50, 75, 95], axis=0)

plt.figure(figsize=(10, 5))
plt.plot(mean_abs_diff, label="Mean Difference (|DQN| - |Delta|)")
for i, p in enumerate([5, 25, 50, 75, 95]):
    plt.plot(percentiles_diff[i], label=f"{p}th percentile")
plt.xlabel("Time Step")
plt.ylabel("Difference in Absolute PnL")
plt.title("Difference in Absolute PnL (DQN - Delta) Over Time")
plt.legend()
plt.show()

### Metrics to be Mesaured

In [ ]:
std_pi_t_delta= np.std(pi_t_delta, axis = 1)
n = pi_t_delta.shape[1]

t_statistic_delta = np.sqrt(n)*(np.mean(pi_t_delta, axis = 1))/std_pi_t_delta

std_pi_t_dqn= np.std(pi_t_dqn, axis = 1)
n = pi_t_dqn.shape[1]

t_statistic_dqn= np.sqrt(n)*(np.mean(pi_t_dqn, axis = 1))/std_pi_t_dqn

# Plot the distribution of pi_t
plt.figure(figsize=(10, 5))
sns.kdeplot(t_statistic_delta, color="orange", label="Student t-stat of Total PnL Delta policy", bw_adjust=3)
sns.kdeplot(t_statistic_dqn, color="green", label="Student t-stat of Total PnL DQN policy", bw_adjust=3)
plt.xlabel("Student t-statistic of Total PnL")
plt.ylabel("Density")
plt.title("Density Plot Student t-statistic of Total PnL")



plt.legend()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Student_t_statistic.png')
plt.show()

In [ ]:
# Compute per-step changes in PnL
pnl_diff_delta = np.diff(pi_t_delta, axis=1)  # Shape (10000, T-1)

# Compute realized volatility per simulation
realized_volatility_delta = np.std(pi_t_delta, axis=1)

# Compute per-step changes in PnL
pnl_diff_dqn = np.diff(pi_t_dqn, axis=1)  # Shape (10000, T-1)

# Compute realized volatility per simulation
realized_volatility_dqn= np.std(pi_t_dqn, axis=1)

# Plot density of realized volatility
plt.figure(figsize=(8, 5))
sns.kdeplot(realized_volatility_delta, color="orange", label="Realized Volatility of Total PnL Delta")
sns.kdeplot(realized_volatility_dqn, color="green", label="Realized Volatility of Total PnL DQN")
plt.xlabel("Realized Volatility of Total PnL")
plt.ylabel("Density")
plt.title("Density Plot of Realized Volatility of Total PnL")

# Set x-axis ticks every 5 steps
x_ticks = np.arange(0, np.max(realized_volatility_dqn)+5, 5)  # Adjust range as needed
plt.xticks(x_ticks)
plt.legend()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Realized_Volatility.png')
plt.show()


In [ ]:
# ...existing code...

# Find indices where delta outperforms DQN
better_delta_idx = np.where(total_pnl_delta > total_pnl_dqn)[0]

# Extract realized volatility for those episodes
vol_delta_wins_dqn = realized_volatility_dqn[better_delta_idx]
vol_delta_wins_delta = realized_volatility_delta[better_delta_idx]

# Plot the distribution of realized volatility for these episodes
plt.figure(figsize=(10, 5))
sns.kdeplot(vol_delta_wins_dqn, label="DQN Realized Volatility (Delta Wins)", color="green", shade = False)
sns.kdeplot(vol_delta_wins_delta, label="Delta Realized Volatility (Delta Wins)", color="orange", shade = False)
plt.xlabel("Realized Volatility")
plt.ylabel("Density")
plt.title("Realized Volatility When Delta Outperforms DQN")
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Realized_Volatility_Delta_Wins.png')
plt.legend()
plt.show()

# Optionally, print summary statistics
print("Mean DQN volatility (Delta wins):", np.mean(vol_delta_wins_dqn))
print("Mean Delta volatility (Delta wins):", np.mean(vol_delta_wins_delta))

In [ ]:
# For episodes where delta wins, compute the mean cumulative PnL difference over time
cum_pnl_diff = np.cumsum(pi_t_delta[better_delta_idx] - pi_t_dqn[better_delta_idx], axis=1)
mean_cum_pnl_diff = np.mean(cum_pnl_diff, axis=0)

plt.figure(figsize=(10, 5))
plt.plot(mean_cum_pnl_diff, label="Mean Cumulative PnL Difference (Delta - DQN)", color="blue")
plt.xlabel("Time Step")
plt.ylabel("Mean Cumulative PnL Difference")
plt.title("Cumulative PnL Difference Over Time (Delta Wins Episodes)")
plt.legend()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Cumulative_PnL_Difference.png')
plt.show()

In [ ]:
# For episodes where delta wins, compute cumulative PnL difference over time
cum_pnl_diff = np.cumsum( pi_t_dqn[better_delta_idx]-pi_t_delta[better_delta_idx] , axis=1)

# Compute percentiles at each time step
percentiles = [5, 25, 50, 75, 95]
cum_pnl_percentiles = np.percentile(cum_pnl_diff, percentiles, axis=0)

plt.figure(figsize=(10, 5))
# Shade between 5th and 95th percentile (light blue)
plt.fill_between(range(cum_pnl_percentiles.shape[1]), cum_pnl_percentiles[0], cum_pnl_percentiles[-1], color='#ADD8E6', alpha=0.5, label='5th-95th percentile')
# Shade between 25th and 75th percentile (light yellow)
plt.fill_between(range(cum_pnl_percentiles.shape[1]), cum_pnl_percentiles[1], cum_pnl_percentiles[3], color='#FFFACD', alpha=0.7, label='25th-75th percentile')
# Median line (bright red)
plt.plot(cum_pnl_percentiles[2], color='blue', linestyle='-', linewidth=2, label='Median (50th percentile)')
# Percentile lines
plt.plot(cum_pnl_percentiles[0], color='#1E90FF', linestyle='--')
plt.plot(cum_pnl_percentiles[1], color='#FFD700', linestyle='--')
plt.plot(cum_pnl_percentiles[3], color='#FFD700', linestyle='--')
plt.plot(cum_pnl_percentiles[4], color='#1E90FF', linestyle='--')

plt.xlabel("Time Step")
plt.ylabel("Cumulative PnL Difference (DQN - Delta PnL)")
plt.title("Percentiles of Cumulative PnL Difference Over Time (DQN Underperforms Delta)")
plt.legend()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Cumulative_PnL_Difference_Percentiles.png')
plt.show()

In [ ]:
# For episodes where delta wins, compute cumulative PnL difference over time
random_indices = np.random.choice(better_dqn_idx, size=len(better_delta_idx), replace=True)

cum_pnl_diff = np.cumsum(pi_t_dqn[random_indices] -pi_t_delta[random_indices] , axis=1)

# Compute percentiles at each time step
percentiles = [5, 25, 50, 75, 95]
cum_pnl_percentiles = np.percentile(cum_pnl_diff, percentiles, axis=0)

plt.figure(figsize=(10, 5))
# Shade between 5th and 95th percentile (light blue)
plt.fill_between(range(cum_pnl_percentiles.shape[1]), cum_pnl_percentiles[0], cum_pnl_percentiles[-1], color='#ADD8E6', alpha=0.5, label='5th-95th percentile')
# Shade between 25th and 75th percentile (light yellow)
plt.fill_between(range(cum_pnl_percentiles.shape[1]), cum_pnl_percentiles[1], cum_pnl_percentiles[3], color='#FFFACD', alpha=0.7, label='25th-75th percentile')
# Median line (bright red)
plt.plot(cum_pnl_percentiles[2], color='blue', linestyle='-', linewidth=2, label='Median (50th percentile)')
# Percentile lines
plt.plot(cum_pnl_percentiles[0], color='#1E90FF', linestyle='--')
plt.plot(cum_pnl_percentiles[1], color='#FFD700', linestyle='--')
plt.plot(cum_pnl_percentiles[3], color='#FFD700', linestyle='--')
plt.plot(cum_pnl_percentiles[4], color='#1E90FF', linestyle='--')

plt.xlabel("Time Step")
plt.ylabel("Cumulative PnL Difference (DQN - Delta PnL)")
plt.title("Percentiles of Cumulative PnL Difference Over Time (DQN outperforms Delta)")
plt.legend()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Cumulative_PnL_Difference_Percentiles_DQN_Wins.png')
plt.show()

In [ ]:
# Extract gamma profiles for episodes where delta wins
gamma_delta_wins = env.gamma_path[better_delta_idx, :]  # shape: (num_delta_wins, T)
gamma_delta_lose = env.gamma_path[random_indices, :]  # shape: (num_delta_wins, T)

# Plot mean gamma profile for these episodes
mean_gamma_delta_wins = np.mean(gamma_delta_wins, axis=0)
mean_gamma_delta_lose = np.mean(gamma_delta_lose, axis=0)
plt.figure(figsize=(10, 5))
plt.plot(mean_gamma_delta_wins, label="Mean Gamma (Delta has higher total PnL)", color="purple")
plt.plot(mean_gamma_delta_lose, label="Mean Gamma (DQN has higher total PnL)", color="gray", linestyle="--")
plt.xlabel("Time Step")
plt.ylabel("Gamma")
plt.title("Mean Gamma Profile When Delta Outperforms DQN")
plt.legend()
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Mean_Gamma_Delta_Wins.png')
plt.show()

In [ ]:
# Extract stock paths for episodes where DQN outperforms Delta
better_dqn_idx = np.where(total_pnl_dqn > total_pnl_delta)[0]
stock_paths_dqn_wins = env.path[better_dqn_idx]

# Plot stock paths for a few sample episodes
plt.figure(figsize=(12, 6))
for i in range(min(1, len(better_dqn_idx))):  # Plot up to 5 sample paths
    plt.plot(stock_paths_dqn_wins[i], label=f'Episode {better_dqn_idx[i]}')

plt.xlabel("Time Step")
plt.ylabel("Stock Price")
plt.title("Stock Paths for Episodes Where DQN Outperforms Delta")
plt.grid(True)
plt.show()

In [ ]:
# Select episodes where delta wins
better_delta_idx = np.where(total_pnl_delta > total_pnl_dqn)[0]

# Extract DQN rewards and actions for these episodes
rewards_delta_wins = [episode_rewards[i] for i in better_delta_idx]
actions_delta_wins = dqn_actions[better_delta_idx]

# Plot reward trajectories for a few sample episodes
plt.figure(figsize=(12, 6))
for i in range(min(5, len(better_delta_idx))):
    plt.plot(rewards_delta_wins[i], label=f'Episode {better_delta_idx[i]}')
plt.xlabel("Time Step")
plt.ylabel("DQN Reward")
plt.title("Reward Trajectories for DQN (Delta Wins Episodes)")
plt.legend()
plt.show()


In [ ]:
# Select episodes where delta wins
better_dqn_idx = np.where(total_pnl_delta < total_pnl_dqn)[0]

# Extract DQN rewards and actions for these episodes
rewards_dqn_wins = [episode_rewards[i] for i in better_dqn_idx]
actions_dqn_wins = dqn_actions[better_dqn_idx]

# Plot reward trajectories for a few sample episodes
plt.figure(figsize=(12, 6))
for i in range(min(5, len(better_dqn_idx))):
    plt.plot(rewards_dqn_wins[i], label=f'Episode {better_dqn_idx[i]}')
plt.xlabel("Time Step")
plt.ylabel("DQN Reward")
plt.title("Reward Trajectories for DQN (DQN Wins Episodes)")
plt.legend()
plt.show()

### Comparing delta hegding path with DQN Path 

In [ ]:
import sys
bs_delta = env.delta_path*100
delta_path = np.insert(bs_delta, 0, 0, axis=1)
delta_h = np.diff(delta_path, axis=1)
stock_path = env.path


for eps in range(73, 79, 2):
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))

    # Subplot 1: Stock price
    axes[0].plot(stock_path[eps], label="stock price")
    axes[0].set_title(f"Stock Price (Episode {eps})")
    axes[0].set_xlabel("Time Step")
    axes[0].set_ylabel("Price")
    axes[0].grid(True)
    axes[0].legend()

    # Subplot 2: Delta hedge vs DQN action
    axes[1].plot(delta_path[eps][1:], label="delta hedge", color="orange")
    axes[1].plot(dqn_actions[eps], label="dqn action",  color="green")
    axes[1].set_xlabel("Time Step")
    axes[1].set_ylabel("Stock position")
    axes[1].set_title(f"Delta vs DQN Action (Episode {eps})")
    axes[1].set_xlim(1, len(dqn_actions[eps]))
    axes[1].grid(True)
    axes[1].legend()

    #Subplot 3: Total pnl
    axes[2].plot(pi_t_delta[eps], label="Total PnL Delta", color="orange")
    axes[2].plot(pi_t_dqn[eps], label="Total PnL DQN", color="green")
    axes[2].set_title(f"Total PnL (Episode {eps})")
    axes[2].set_xlabel("Time Step")
    axes[2].set_ylabel("Total PnL")
    axes[2].set_xlim(1, len(dqn_actions[eps]))
    axes[2].text(0.02,0.55,
                  f"Total PnL Delta: {total_pnl_delta[eps]:.2f}\nTotal PnL DQN: {total_pnl_dqn[eps]:.2f}", 
                  transform=axes[2].transAxes,
                  fontsize=12, 
                  verticalalignment='top',
                  bbox=dict(boxstyle='round', edgecolor='black', facecolor='white', alpha=0.7))
    axes[2].grid(True)
    axes[2].legend()

    # #Subplot 4: Reward Tracking 
    # axes[3].plot(episode_rewards[eps], label="dqn reward",  color="green")
    # axes[3].set_xlabel("Time Step")
    # axes[3].set_ylabel("Reward")
    # axes[3].set_title(f"Episode Rewards {eps})")
    # axes[3].grid(True)
    # axes[3].legend()


    plt.tight_layout()
    plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Stock_and_Action_TotalPnl_{eps}.png')
    plt.show()


In [ ]:
eps = 21
plt.figure(figsize=(10, 5))
plt.plot(-delta_path[eps][1:-1], label="delta hedge", color="orange")
plt.plot(-dqn_actions[eps][1:], label="dqn action",  color="#6495ED")
plt.plot(v_t_diff[eps], label="Option PnL", color="#F08080")
plt.plot(-h_t_dqn[eps], label="Stock PnL ", color="green")
plt.plot(pi_t_dqn[eps], label="Total PnL", color="black", linestyle="--")
plt.xlabel("Time Step")
plt.ylabel("PnL")
plt.title("Out of Sample Simulation DQN") 
plt.legend()
plt.grid(True)
plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_OOS_Simulation_{eps}.png')
plt.show

### Policy Plots 

In [ ]:
ttm_array = [10, 5, 0.2]

for ttm in ttm_array:

    dqn_otm = []
    dqn_atm = []
    dqn_itm = []

    price_otm = 80
    price_atm = 100
    price_itm =120
    strike = 100

    stock_positions = np.arange(20,101,1)

    for stock_position in stock_positions: 
        state_otm = [price_otm, stock_position, ttm]
        action_otm= dqn.get_action(state_otm)
        dqn_otm.append(action_otm-stock_position)

        state_itm = [price_itm, stock_position, ttm]
        action_itm= dqn.get_action(state_itm)
        dqn_itm.append(action_itm-stock_position)

        state_atm = [price_atm, stock_position, ttm]
        action_atm= dqn.get_action(state_atm)
        dqn_atm.append(action_atm-stock_position)


    delta_itm= []
    delta_atm = [] 
    delta_otm = []

    stock_pos= np.arange(-20, -101, -1)

    for i, pos in enumerate(stock_pos):
        delta_itm.append(-100-pos)
        delta_atm.append(-50-pos)
        delta_otm.append(-pos)

    light_red = "#F08080"
    light_blue = "#6495ED"

    plt.figure(figsize=(10, 5))
    plt.scatter(-stock_positions, -np.array(dqn_otm), label="OTM DQN", color=light_red, marker='*' )
    plt.scatter(-stock_positions, -np.array(dqn_atm), label="ATM DQN", color=light_blue)
    plt.scatter(-stock_positions, -np.array(dqn_itm), label="ITM DQN", color="green", marker = 'D')
    plt.plot(-stock_positions, delta_itm, label="ITM Delta", color="green", linestyle = "dashdot")
    plt.plot(-stock_positions,delta_atm, label="ATM Delta", color=light_blue, linestyle = "dashed")
    plt.plot(-stock_positions,delta_otm, label="OTM Delta", color=light_red, linestyle = "dotted")
    plt.xlim(-10, -105)
    # plt.text(0, 80, "Long", fontsize=14, color="black",)  # Positive side
    # plt.text(-5, -80, "Short", fontsize=14, color="black",) 
    plt.xlabel("stock positions")
    plt.ylabel("Actions", fontsize = 14)
    plt.title("Policy Plot")
    plt.grid(True)
    plt.legend()
    plt.savefig(f'history/dqn/Training{train_num}/Training{train_num}_Policy_Plot_{ttm}.png')
    plt.show()




In [49]:
dqn_actions

array([[  0,  51,  57, ...,   1,   0,   0],
       [  0,  51,  44, ..., 100, 100, 100],
       [  0,  51,  53, ...,  99, 100,  98],
       ...,
       [  0,  51,  53, ...,  33,  45,   0],
       [  0,  51,  57, ...,  18,  45,  98],
       [  0,  51,  53, ...,  12,   1,   1]], dtype=int64)

In [ ]:
# Flatten everything to 1D arrays
dqn_flat = dqn_actions.flatten()
delta_flat = delta_path.flatten()

plt.figure(figsize=(7, 6))
plt.scatter(delta_flat, dqn_flat, alpha=0.25)

# Add 45° reference line
min_val = min(delta_flat.min(), dqn_flat.min())
max_val = max(delta_flat.max(), dqn_flat.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', label="Perfect Match")

plt.xlabel("Delta Hedge")
plt.ylabel("DQN Hedge")
plt.title("Scatter: DQN Hedge vs Delta Hedge (Aggregated Across All Paths)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

TypeError: 'tuple' object is not callable